In [84]:
import pandas as pd
import json
from collections import defaultdict
import re

#### Con este bucle obtenemos los datos sobre la inicialización de todos los niveles jugados. PRUEBA

Si se han usado los pasos minimos:

In [85]:
def extraerPasosMinimos(rawData):
    erLevel = re.compile(r'\blevel$\b')
    erCompleted = re.compile(r'\bcompleted$\b')
    erIdLevel = re.compile(r'/')
    
    data = defaultdict(defaultdict)

    for evento in rawData:
        verb = evento["verb"]["id"]
        obj = evento["object"]["definition"]["type"]
        if erLevel.search(obj) : #Si el objeto de la acción es un nivel
            name = evento["actor"]["name"]
            levelCode = erIdLevel.split(evento["object"]["id"])[-1]
            
            if erCompleted.search(verb):
                if evento["result"]["score"]["raw"] > 0 :
                    if evento["result"]["success"]: #Significa que el nivel se ha completado con éxito
                        if levelCode in data[name]:
                            #print("dghfthrh")
                            data[name][levelCode].append([{"pasosMinimos" : evento["result"]["extensions"]["minimum_steps"]}]) 
                        else: 
                            #print(evento)
                            data[name][levelCode]= [{"pasosMinimos" : evento["result"]["extensions"]["minimum_steps"]}]
                    else:
                        if levelCode in data[name]:
                            data[name][levelCode]= [{"pasosMinimos" : None}]
                

    return data            

Si se han usado pistas:

In [86]:
def extraerPistas(rawData):
    erLevel = re.compile(r'\blevel$\b')
    erCompleted = re.compile(r'\bcompleted$\b')
    erIdLevel = re.compile(r'/')
    
    data = defaultdict(defaultdict)

    for evento in rawData:
        verb = evento["verb"]["id"]
        obj = evento["object"]["definition"]["type"]
        if erLevel.search(obj) : #Si el objeto de la acción es un nivel
            name = evento["actor"]["name"]
            levelCode = erIdLevel.split(evento["object"]["id"])[-1]
            
            if erCompleted.search(verb):
                if evento["result"]["score"]["raw"] > 0 :
                    if evento["result"]["success"]: #Significa que el nivel se ha completado con éxito
                        if levelCode in data[name]:
                            #print("dghfthrh")
                            data[name][levelCode].append([{"pistasNoUsadas" : evento["result"]["extensions"]["no_hints"]}]) 
                        else: 
                            #print(evento)
                            data[name][levelCode]= [{"pistasNoUsadas" : evento["result"]["extensions"]["no_hints"]}]
                    else:
                        if levelCode in data[name]:
                            data[name][levelCode]= [{"pistasNoUsadas" : None}]
                

    return data            

Si esta hecho en primera ejecucion:

In [87]:
def extraerPrimeraEjecucion(rawData):
    erLevel = re.compile(r'\blevel$\b')
    erCompleted = re.compile(r'\bcompleted$\b')
    erIdLevel = re.compile(r'/')
    
    data = defaultdict(defaultdict)

    for evento in rawData:
        verb = evento["verb"]["id"]
        obj = evento["object"]["definition"]["type"]
        if erLevel.search(obj) : #Si el objeto de la acción es un nivel
            name = evento["actor"]["name"]
            levelCode = erIdLevel.split(evento["object"]["id"])[-1]
            
            if erCompleted.search(verb):
                if evento["result"]["score"]["raw"] > 0 :
                    if evento["result"]["success"]: #Significa que el nivel se ha completado con éxito
                        if levelCode in data[name]:
                            #print("dghfthrh")
                            data[name][levelCode].append([{"primeraEjecucion" : evento["result"]["extensions"]["first_execution"]}]) 
                        else: 
                            #print(evento)
                            data[name][levelCode]= [{"primeraEjecucion" : evento["result"]["extensions"]["first_execution"]}]
                    else:
                        if levelCode in data[name]:
                            data[name][levelCode]= [{"primeraEjecucion" : None}]
                

    return data            

Cuantas estrellas se han sacado:

In [88]:
def extraerEstrellas(rawData):
    erLevel = re.compile(r'\blevel$\b')
    erCompleted = re.compile(r'\bcompleted$\b')
    erIdLevel = re.compile(r'/')
    
    data = defaultdict(defaultdict)

    for evento in rawData:
        verb = evento["verb"]["id"]
        obj = evento["object"]["definition"]["type"]
        if erLevel.search(obj) : #Si el objeto de la acción es un nivel
            name = evento["actor"]["name"]
            levelCode = erIdLevel.split(evento["object"]["id"])[-1]
            
            if erCompleted.search(verb):
                if evento["result"]["score"]["raw"] > 0 :
                    if evento["result"]["success"]: #Significa que el nivel se ha completado con éxito
                        if levelCode in data[name]:
                            #print("dghfthrh")
                            data[name][levelCode].append([{"estrellas" : evento["result"]["score"]["raw"]}]) 
                        else: 
                            #print(evento)
                            data[name][levelCode]= [{"estrellas" : evento["result"]["score"]["raw"]}]
                    else:
                        if levelCode in data[name]:
                            data[name][levelCode]= [{"estrellas" : None}]
                

    return data            

Si se han conseguido las 3 estrellas o no:

In [89]:
def extraer3Estrellas(rawData):
    erLevel = re.compile(r'\blevel$\b')
    erCompleted = re.compile(r'\bcompleted$\b')
    erIdLevel = re.compile(r'/')
    
    data = defaultdict(defaultdict)

    for evento in rawData:
        verb = evento["verb"]["id"]
        obj = evento["object"]["definition"]["type"]
        if erLevel.search(obj) : #Si el objeto de la acción es un nivel
            name = evento["actor"]["name"]
            levelCode = erIdLevel.split(evento["object"]["id"])[-1]
            
            if erCompleted.search(verb):
                if evento["result"]["score"]["raw"] > 0 :
                    if evento["result"]["success"]: #Significa que el nivel se ha completado con éxito
                        if levelCode in data[name]:
                            #print("dghfthrh")
                            if evento["result"]["score"]["raw"] == 3:
                                data[name][levelCode].append([{"3 estrellas" : "Conseguidas"}]) 
                            else:
                                data[name][levelCode].append([{"3 estrellas" : "No Conseguidas"}])
                        else: 
                            #print(evento)
                            if evento["result"]["score"]["raw"] == 3:
                                data[name][levelCode]= [{"3 estrellas" : "Conseguidas"}]
                            else:
                                data[name][levelCode]= [{"3 estrellas" : "No Conseguidas"}]
                    else:
                        if levelCode in data[name]:
                            data[name][levelCode]= [{"3 estrellas" : None}]
                

    return data            

In [93]:
JSONFile = open('traces_Articoding_Escolapias.json')
rawData = json.load(JSONFile)
data = extraerPasosMinimos(rawData)
pd.DataFrame(data)

#data = extraerPistas(rawData)
#pd.DataFrame(data)

#data = extraerPrimeraEjecucion(rawData)
#pd.DataFrame(data)

#data = extraerEstrellas(rawData)
#pd.DataFrame(data)

#data = extraer3Estrellas(rawData)
#pd.DataFrame(data)

JSONFile.close()

pd.options.display.max_columns = None
pd.set_option('display.max_colwidth', None)
#pd.DataFrame(data)

pd.DataFrame(data)

,gqoj,hgfx,uala,qeqyj,fkmtp,eprdu,bdwyj,pbgky,hjpbw,ftpnb,loemd,gcpze,gleqe,srjon,khadt,sxipu,ikcar,uzzpo,vsxld,zxvkj,ejeuw,kmqou,xnpqf,ipdcp,epqkm,zwzfv,rdiaf,auvqa,krahz,grkij,zcibf,ubmtx,pygoc,onyep,cmbfi,asopk,yvpmf,oahey,edxmq,bhyhw,accul,tekwp,swvxx,dunul,kkoif,yfohf,dwqwi,tjfzy,nmveq,sjpkc,lwryl,fivin,ewqbs,tinco,psido,zbgwk,fyyya,gdljy,mymho,wjaxi,yxewz,gguuc,mjiso,hpdxj,dvmlu,camhi,kkliq,hqmhq,tgwmn,rziia,ucdvy,dmwvy,ztyed,yxjji,qfsdj,stiry,bjvnt,ngavz,vddlg,atlzs,yazod,tqgyp,idmzi,huyyb,zdnso,lhrbj,tavin,vvqlm,aotlm,mgaak,wqmgq,knewj,tvxow,ahxym,rqlug,jozyx,xtdrq,jhbuv,ptmkr,pzfdd,buvgs,sbexy,thaho,pbwkf,khfha,mewpz,fmtrq,icllt,nhbnj,qwnpf,itkas,uniee,esrit,rqaby,yyvas,kzles,gzgfk,hcbyl,rkmrw,nuasx,nplrf,ixrux,pqwaf,cojqb,qgcgc,lohyo,rfofs,qbsjv,rsjxu,sybwc,eayrh,orjyn,bajbm,fbldj,tcvyz,blvkx,tflzv,raqdt,pleui,xbmmw,prpej,vkfnt,fmupf,olugf,epjcx
tutorials_1,"[{'pistasUsadas': True}, [{'pistasUsadas': True}], [{'pistasUsadas': True}]]",[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],"[{'pistasUsadas': True}, [{'pistasUsadas': True}]]",[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsadas': True}],[{'pistasUsad